# Boto 3 for S3 Tutorial in Python

![Miro Image](https://miro.medium.com/1*fFXpjwt3uJgMrUTKip24dw.png)

# List S3 Buckets and Create an S3 Bucket 

In [14]:
import boto3
s3 = boto3.client('s3')

bucket_name = 'randomtestabdullah'

In [15]:
# Get all buckets 

response = s3.list_buckets()
response['Buckets']

[]

In [16]:
# Create S3 Bucket

def create_bucket(bucket_name: str):
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': 'eu-north-1'
        }
    )
    print(f"Bucket \" {bucket_name} \" is created")

# Test
create_bucket(bucket_name)

Bucket " randomtestabdullah " is created


# Upload a Single File to S3

In [17]:
import os

def upload_file(bucket_name:str, file_path: str, object_name:str = None):
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3.upload_file(file_path, bucket_name, object_name)

In [18]:
filepath = "assets/index.html"
upload_file(bucket_name, filepath)

In [25]:
# Generating Random data 
for i in range(5):
    upload_file(bucket_name, file_path=filepath, object_name=f"test {i}")

# List all objects in a bucket

In [37]:
def list_objects(bucket_name):
    
    """List all the object in the bucket"""
    
    response = s3.list_objects_v2(Bucket = bucket_name)
    objects = response['Contents']
    
    for i in range(len(objects)) :
        print(objects[i]['Key'])

list_objects(bucket_name)

index.html
test 0
test 1
test 2
test 3
test 4


# Download S3 File to Local System

In [44]:
def download_file(Bucket: str, ObjName: str, filePath: str):
    if not os.path.exists(os.path.dirname(filePath)):
        os.makedirs(filePath)

    s3.download_file(Bucket, ObjName, filePath)

for i in range (5):    
    download_file(bucket_name, f"test {i}", f"assets/test {i}")

# Upload Folder to S3

In [50]:
def upload_folder(Bucket, dir_path, s3_prefix):
    for root, dirs, files in os.walk("assets"):
        for file in files:
            filepath = os.path.join(root, file)
            s3_key = os.path.join(s3_prefix, file)

            s3.upload_file(filepath, Bucket, s3_key)

            
upload_folder(Bucket= bucket_name ,dir_path="assets", s3_prefix="s3_data")

# Download Folder from S3 

In [63]:
local_path = "s3_assets"
s3_prefix = "s3_data"

def download_folder(bucket_name, local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    
    paginator = s3.get_paginator('list_objects_v2')
    
    for result in paginator.paginate(Bucket = bucket_name, Prefix = s3_prefix):
        if 'Contents' in result:
            for key in result['Contents']:
                s3_key = key["Key"]
                # print(s3_key)
                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                s3.download_file(bucket_name, s3_key, local_file)

download_folder(bucket_name, local_path, s3_prefix)

# Delete All Files in an S3 Bucket

In [72]:
def delete_bucket_files(bucket_name):
    # Retrieve a list of all objects in the bucket
    response = s3.list_objects_v2(Bucket = bucket_name)

    # Check if the bucket contains any files
    if 'Contents' in response:
        # Loop through the list of objects
        for obj in response['Contents']:
            # Delete each object individually by passing it in the 'Objects' list
            s3.delete_objects(
                Bucket = bucket_name,
                Delete = {
                    'Objects': [
                        {
                            'Key': obj["Key"]
                        }
                    ]
                }
            )
            print(f"File {obj['Key']} deleted.")
    else:
        print(f"Bucket '{bucket_name}' is already empty.")

delete_bucket_files(bucket_name)

File index.html deleted.
File s3_data/index.html deleted.
File s3_data/test 0 deleted.
File s3_data/test 1 deleted.
File s3_data/test 2 deleted.
File s3_data/test 3 deleted.
File s3_data/test 4 deleted.
File test 0 deleted.
File test 1 deleted.
File test 2 deleted.
File test 3 deleted.
File test 4 deleted.


# Delete S3 Bucket
Must delete all files before this step

In [74]:
s3.delete_bucket(Bucket = bucket_name)

{'ResponseMetadata': {'RequestId': 'NTTJAB4V3QD04YAQ',
  'HostId': 'BY7hoCWTRlOSA7BtgNwCV/qIKvznuL02ilWAEtyxEtNz4ni9WH3I0Zu9Zi4hfKaN3yvi3Uh2DjMHYBUCVkHnupGuas2ErjhwEwFrdCsBCGA=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'BY7hoCWTRlOSA7BtgNwCV/qIKvznuL02ilWAEtyxEtNz4ni9WH3I0Zu9Zi4hfKaN3yvi3Uh2DjMHYBUCVkHnupGuas2ErjhwEwFrdCsBCGA=',
   'x-amz-request-id': 'NTTJAB4V3QD04YAQ',
   'date': 'Sat, 30 Aug 2025 16:28:31 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}